In [2]:
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
from tqdm.auto import tqdm
import torch.nn as nn
import torch.nn.functional as F
from ase.db import connect
from src import data_proc

# 设置设备
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 定义 Schmidt 正交化函数（用于处理目标值）
def schmidt_orthogonalization(vectors):
    vectors = vectors.reshape(3, 3)
    diagonal_elements = np.diagonal(vectors)
    ele = np.mean(diagonal_elements)
    return ele

# 定义 MLP 模型
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 自定义数据集类
class CrystalDataset(Dataset):
    def __init__(self, db_path):
        self.db = connect(db_path)
        self.entries = list(self.db.select())

    def __len__(self):
        return len(self.entries)

    def __getitem__(self, idx):
        tmp = self.entries[idx]
        atoms = tmp.toatoms()
        try:
            atom_feature = data_proc.get_crystal_path_muhead(ase_obj=True, stru=atoms, num_heads=1)
            target = torch.tensor(tmp.data['dielectric'], dtype=torch.float32)  # 保留目标属性
            atom_feature = torch.tensor(atom_feature, dtype=torch.float32)
            return atom_feature, target  # 返回晶体特征和目标属性
        except ValueError as e:
            print(f"Skipping structure {idx} due to error: {e}")
            return None

# 自定义 collate 函数
def collate_fn(batch):
    batch = [b for b in batch if b is not None]
    if len(batch) == 0:
        return None

    atom_features, targets = zip(*batch)
    num_heads = atom_features[0].shape[0]
    max_atoms = max(feat.shape[1] for feat in atom_features)
    batch_size = len(atom_features)
    embed_dim = atom_features[0].shape[2]

    # 填充特征并展平 num_heads 维度
    padded_features = torch.zeros((batch_size * num_heads, max_atoms, embed_dim), dtype=torch.float32)
    attention_masks = torch.zeros((batch_size * num_heads, max_atoms), dtype=torch.float32)
    flattened_targets = torch.zeros((batch_size * num_heads, *targets[0].shape), dtype=torch.float32)

    for i, (feat, target) in enumerate(zip(atom_features, targets)):
        num_atoms = feat.shape[1]
        for head in range(num_heads):
            idx = i * num_heads + head
            padded_features[idx, :num_atoms, :] = feat[head]
            attention_masks[idx, :num_atoms] = 1
            flattened_targets[idx] = target

    return padded_features, attention_masks, flattened_targets

# 定义 GRU 编码器
class GRUEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=2):
        super(GRUEncoder, self).__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        _, h_n = self.gru(x)  # h_n: (num_layers, batch_size, hidden_dim)
        h_n = h_n[-1]  # 取最后一层的隐藏状态
        output = self.fc(h_n)
        return output

class ContrastiveModel(nn.Module):
    def __init__(self, feature_dim, hidden_dim, output_dim):
        super(ContrastiveModel, self).__init__()
        self.encoder = GRUEncoder(feature_dim, hidden_dim, output_dim)
        self.logit_scale = nn.Parameter(torch.ones([]) * np.log(1 / 0.07))

    def forward(self, x1, x2=None):
        if x2 is None:
            return self.encoder(x1)
        z1 = self.encoder(x1)
        z2 = self.encoder(x2)
        return z1, z2
# 加载预训练模型
feature_dim = 384
hidden_dim = 256
output_dim = 128
model = ContrastiveModel(feature_dim, hidden_dim, output_dim).to(device)
model.load_state_dict(torch.load("best_path_feature.pth"))
model.eval()

# 加载数据集
dataset = CrystalDataset("/data/home/hzw1010/suth/elec_gw/dbs/clean.db")
dataloader = DataLoader(dataset, batch_size=64, num_workers=2, pin_memory=False,
                        shuffle=False, collate_fn=collate_fn, drop_last=False)

# 提取特征
def extract_features(model, dataloader, device):
    features = []
    targets = []
    with torch.no_grad():
        for batch_features, batch_masks, batch_targets in tqdm(dataloader, desc="Extracting Features"):
            if batch_features is None:
                continue
            batch_features = batch_features.to(device)
            z = model(batch_features)  # 提取特征
            features.append(z.cpu())
            targets.append(batch_targets.cpu())
    return torch.cat(features, dim=0), torch.cat(targets, dim=0)

# 处理目标值
def process_targets(targets):
    processed_targets = []
    for target in targets:
        elec = schmidt_orthogonalization(target.numpy())
        processed_targets.append(elec)
    return torch.tensor(processed_targets, dtype=torch.float32)

# 提取特征并处理目标值
features, targets = extract_features(model, dataloader, device)
processed_targets = process_targets(targets)

Using device: cuda:2


/tmp/ipykernel_2030625/89095818.py:115: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_path_feature.pth"))


Extracting Features:   0%|          | 0/114 [00:00<?, ?it/s]

In [9]:
import pandas as pd
from autogluon.tabular import TabularPredictor
dataset_size = len(features)
train_size = int(0.8 * dataset_size)  
train_data = pd.DataFrame(features[:train_size].numpy())
train_labels = pd.Series(processed_targets[:train_size].numpy())
val_data = pd.DataFrame(features[train_size:].numpy())
val_labels = pd.Series(processed_targets[train_size:].numpy())
predictor = TabularPredictor(label="target", eval_metric="mean_squared_error")
train_data['target'] = train_labels

No path specified. Models will be saved in: "AutogluonModels/ag-20250116_060244"


In [10]:
predictor.fit(
    train_data, 
    presets="best_quality",
    time_limit=36000
)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.9.21
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Tue Jun 1 16:14:33 UTC 2021
CPU Count:          96
Memory Avail:       234.09 GB / 251.30 GB (93.2%)
Disk Space Avail:   529624.39 GB / 596060.00 GB (88.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacked overfit

(_ray_fit pid=2100138) [1000]	valid_set's l2: 21.1798
(_ray_fit pid=2100138) [2000]	valid_set's l2: 21.1433


(_dystack pid=2088280) 	-23.6093	 = Validation score   (-mean_squared_error)
(_dystack pid=2088280) 	5.86s	 = Training   runtime
(_dystack pid=2088280) 	0.04s	 = Validation runtime
(_dystack pid=2088280) Fitting model: LightGBM_BAG_L1 ... Training model for up to 5959.26s of the 8955.03s of remaining time.
(_dystack pid=2088280) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=12, gpus=0, memory=0.02%)
(_dystack pid=2088280) 	-25.0074	 = Validation score   (-mean_squared_error)
(_dystack pid=2088280) 	3.17s	 = Training   runtime
(_dystack pid=2088280) 	0.02s	 = Validation runtime
(_dystack pid=2088280) Fitting model: RandomForestMSE_BAG_L1 ... Training model for up to 5935.10s of the 8930.87s of remaining time.
(_dystack pid=2088280) 	-27.1788	 = Validation score   (-mean_squared_error)
(_dystack pid=2088280) 	5.03s	 = Training   runtime
(_dystack pid=2088280) 	0.31s	 = Validation runtime
(_dystack pid=2088280) Fitting model: C

(_ray_fit pid=2131446) [1000]	valid_set's l2: 27.2697
(_ray_fit pid=2131453) [1000]	valid_set's l2: 24.2149
(_ray_fit pid=2131446) [2000]	valid_set's l2: 27.2692


(_dystack pid=2088280) 	-27.3011	 = Validation score   (-mean_squared_error)
(_dystack pid=2088280) 	22.74s	 = Training   runtime
(_dystack pid=2088280) 	0.06s	 = Validation runtime
(_dystack pid=2088280) Fitting model: CatBoost_r177_BAG_L1 ... Training model for up to 5526.50s of the 8522.27s of remaining time.
(_dystack pid=2088280) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=12, gpus=0, memory=0.03%)
(_dystack pid=2088280) 	-24.8064	 = Validation score   (-mean_squared_error)
(_dystack pid=2088280) 	71.9s	 = Training   runtime
(_dystack pid=2088280) 	0.04s	 = Validation runtime
(_dystack pid=2088280) Fitting model: NeuralNetTorch_r79_BAG_L1 ... Training model for up to 5429.52s of the 8425.29s of remaining time.
(_dystack pid=2088280) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=12, gpus=0, memory=0.01%)
(_dystack pid=2088280) 	-19.22	 = Validation score   (

...

(_dystack pid=2088280) 	-23.5106	 = Validation score   (-mean_squared_error)
(_dystack pid=2088280) 	7.98s	 = Training   runtime
(_dystack pid=2088280) 	0.06s	 = Validation runtime
(_dystack pid=2088280) Fitting model: NeuralNetTorch_r143_BAG_L1 ... Training model for up to 1485.20s of the 4480.97s of remaining time.
(_dystack pid=2088280) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=12, gpus=0, memory=0.01%)
(_dystack pid=2088280) 	-18.9871	 = Validation score   (-mean_squared_error)
(_dystack pid=2088280) 	197.8s	 = Training   runtime
(_dystack pid=2088280) 	0.09s	 = Validation runtime
(_dystack pid=2088280) Fitting model: CatBoost_r128_BAG_L1 ... Training model for up to 1251.07s of the 4246.84s of remaining time.
(_dystack pid=2088280) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=12, gpus=0, memory=0.09%)
(_dystack pid=2088280) 	-24.7243	 = Validation score 

(_ray_fit pid=2545618) [1000]	valid_set's l2: 20.5562 [repeated 12x across cluster]
(_ray_fit pid=2545619) [2000]	valid_set's l2: 21.2223 [repeated 8x across cluster]


(_dystack pid=2088280) 	-23.4622	 = Validation score   (-mean_squared_error)
(_dystack pid=2088280) 	17.11s	 = Training   runtime
(_dystack pid=2088280) 	0.12s	 = Validation runtime
(_dystack pid=2088280) Fitting model: XGBoost_r49_BAG_L1 ... Training model for up to 539.37s of the 3535.14s of remaining time.
(_dystack pid=2088280) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=12, gpus=0, memory=0.05%)
(_dystack pid=2088280) 	-25.5289	 = Validation score   (-mean_squared_error)
(_dystack pid=2088280) 	15.4s	 = Training   runtime
(_dystack pid=2088280) 	0.11s	 = Validation runtime
(_dystack pid=2088280) Fitting model: CatBoost_r5_BAG_L1 ... Training model for up to 494.21s of the 3489.98s of remaining time.
(_dystack pid=2088280) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=12, gpus=0, memory=0.01%)
(_dystack pid=2088280) 	-25.4059	 = Validation score   (-mean_squ

KeyboardInterrupt: 

(_dystack pid=2088280) 	-21.1266	 = Validation score   (-mean_squared_error)
(_dystack pid=2088280) 	173.62s	 = Training   runtime
(_dystack pid=2088280) 	0.08s	 = Validation runtime
(_dystack pid=2088280) Fitting model: NeuralNetTorch_r71_BAG_L1 ... Training model for up to 194.05s of the 3189.82s of remaining time.
(_dystack pid=2088280) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=12, gpus=0, memory=0.01%)
(_dystack pid=2088280) 	-21.1389	 = Validation score   (-mean_squared_error)
(_dystack pid=2088280) 	77.52s	 = Training   runtime
(_dystack pid=2088280) 	0.08s	 = Validation runtime
(_dystack pid=2088280) Fitting model: CatBoost_r143_BAG_L1 ... Training model for up to 85.09s of the 3080.86s of remaining time.
(_dystack pid=2088280) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=12, gpus=0, memory=0.05%)
(_dystack pid=2088280) 	Warning: Exception caused CatBo

In [8]:
predictor

In [6]:
# 评估模型性能
val_data['target'] = val_labels
performance = predictor.evaluate(val_data)
print(performance)

{'mean_absolute_error': -1.754349708557129, 'root_mean_squared_error': -5.314374923706055, 'mean_squared_error': -28.242578506469727, 'r2': 0.5191915035247803, 'pearsonr': 0.7241694160215834, 'median_absolute_error': -0.5566833019256592}


In [2]:
mlp_input_dim = output_dim  # 输入维度是特征维度
mlp_hidden_dim = 128
mlp_output_dim = 1  # 输出是一个标量
mlp_model = MLP(mlp_input_dim, mlp_hidden_dim, mlp_output_dim).to(device)

In [ ]:
dataset_size = len(features)
train_size = int(0.8 * dataset_size)  # 80% 训练集
val_size = dataset_size - train_size  # 20% 验证集
train_dataset = TensorDataset(features[:train_size], processed_targets[:train_size])
train_dataset = TensorDataset(features, processed_targets)
val_dataset = TensorDataset(features[train_size:], processed_targets[train_size:])
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)

mlp_input_dim = output_dim  # 输入维度是特征维度
mlp_hidden_dim = 128
mlp_output_dim = 1  # 输出是一个标量
mlp_model = MLP(mlp_input_dim, mlp_hidden_dim, mlp_output_dim).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(mlp_model.parameters(), lr=1e-4)
num_epochs = 2000
from tqdm.auto import tqdm

import logging

# 配置日志记录
logging.basicConfig(filename='training.log', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

best_val_loss = float('inf')

for epoch in range(num_epochs):
    mlp_model.train()
    train_loss = 0.0
    
    # 训练阶段
    with tqdm(train_dataloader, desc=f"Epoch {epoch+1} - Training", unit="batch") as t:
        for batch_features, batch_targets in t:
            batch_features = batch_features.to(device)
            batch_targets = batch_targets.to(device)
            
            optimizer.zero_grad()
            predictions = mlp_model(batch_features)
            loss = criterion(predictions.squeeze(), batch_targets)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            t.set_postfix({'Batch Train Loss': f"{loss.item():.4f}"})
    
    # 验证阶段
    mlp_model.eval()
    val_loss = 0.0
    with tqdm(val_dataloader, desc=f"Epoch {epoch+1} - Validation", unit="batch") as v:
        with torch.no_grad():
            for batch_features, batch_targets in v:
                batch_features = batch_features.to(device)
                batch_targets = batch_targets.to(device)
                predictions = mlp_model(batch_features)
                loss = criterion(predictions.squeeze(), batch_targets)
                val_loss += loss.item()
                v.set_postfix({'Batch Val Loss': f"{loss.item():.4f}"})
    
    # 计算平均损失
    train_loss /= len(train_dataloader)
    val_loss /= len(val_dataloader)
    # 更新训练和验证损失到进度条的描述信息
    #tqdm.write(f"Epoch {epoch+1} - Final Train Loss: {train_loss:.4f}, Final Val Loss: {val_loss:.4f}")
    
    # 保存最佳模型（记录到日志中）
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(mlp_model.state_dict(), "best_mlp_model.pth")
        logging.info(f"Best MLP model saved with Val Loss: {best_val_loss:.4f}")

    # 在下一个 epoch 开始前，更新上一个 epoch 的最终损失信息到进度条中
    if epoch < num_epochs - 1:
        tqdm.write("")  # 空行分隔不同的 epoch 输出

In [ ]:
%config InlineBackend.figure_format = 'svg'
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt
def evaluate_model(model, dataloader, device):
    model.eval()
    all_targets = []
    all_predictions = []
    with torch.no_grad():
        for batch_features, batch_targets in tqdm(dataloader, desc="Evaluating"):
            batch_features = batch_features.to(device)
            batch_targets = batch_targets.to(device)

            predictions = model(batch_features).squeeze()
            all_predictions.append(predictions.cpu())
            all_targets.append(batch_targets.cpu())

    # 合并所有批次数据
    all_predictions = torch.cat(all_predictions).numpy()
    all_targets = torch.cat(all_targets).numpy()

    # 计算 MAE 和 R²
    mae = mean_absolute_error(all_targets, all_predictions)
    r2 = r2_score(all_targets, all_predictions)
    return all_targets, all_predictions, mae, r2
def plot_scatter(targets, predictions):
    plt.figure(figsize=(8, 8))
    plt.scatter(targets, predictions, alpha=0.6, label="Predictions vs Targets")
    plt.plot([min(targets), max(targets)], [min(targets), max(targets)], color="red", linestyle="--", label="Ideal")
    plt.xlabel("True Values")
    plt.ylabel("Predicted Values")
    plt.title(f'Scatter Plot of True vs Predicted Values\nMAE: {mae:.4f}, R²: {r2:.4f}')
    plt.legend()
    plt.grid(True)
    plt.show()
# 加载最佳模型
mlp_model.load_state_dict(torch.load("best_mlp_model.pth"))

# 测试模型性能并绘图
test_targets, test_predictions, mae, r2 = evaluate_model(mlp_model, val_dataloader, device)
print(f"MAE: {mae:.4f}, R²: {r2:.4f}")

# 画散点图
plot_scatter(test_targets, test_predictions)